In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
from dash import ctx
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
# from dash import Dash

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from crud_functions import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "welcomeuser"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

image_filename = 'assets/Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
            href='https://www.snhu.edu',
            target="_blank"
        )
    ),
    html.Center(html.B(html.H1('Austin Animal Center Outcomes Dashboard'))),
    html.Hr(),
    html.Div([
        dcc.Dropdown(
            options=[
                {
                    'label': 'Water Rescue',
                    'value': 'Water Rescue'
                },
                {
                    'label': 'Mountain or Wilderness Rescue',
                    'value': 'Mountain or Wilderness Rescue'
                },
                {
                    'label': 'Disaster Rescue or Individual Tracking',
                    'value': 'Disaster Rescue or Individual Tracking'
                }
            ],
            placeholder='Filter By Rescue Type',
            id='filter-type'
        ),
        html.Button('Reset Filter', id='reset-button', n_clicks=0)
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        style_table={'overflowX': 'auto'},
        data=df.to_dict('records'),
        row_selectable="single",
        selected_rows=[0],
        page_size=25,
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'color': 'white'
        },
        style_data={
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white'
        }
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row', style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                 )
             ]),
    html.Hr(),
    html.P('Developer: Anthony D\'Angelo')
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback(
    Output('datatable-id','data'),
    Output('filter-type','value'),
    [
        Input('filter-type', 'value'),
        Input('reset-button', 'n_clicks')
    ]
)
def update_dashboard(value, n_clicks):
    if value == 'Water Rescue':
        df = pd.DataFrame.from_records(db.water_rescue_filter())
        dropdown_placehoder = value
    if value == 'Mountain or Wilderness Rescue':
        df = pd.DataFrame.from_records(db.mountain_wilderness_rescue_filter())
        dropdown_placehoder = value
    if value == 'Disaster Rescue or Individual Tracking':
        df = pd.DataFrame.from_records(db.disaster_individual_rescue_filter())
        dropdown_placehoder = value
    if 'reset-button' == ctx.triggered_id or value is None:
        df = pd.DataFrame.from_records(db.read({}))
        dropdown_placehoder = None
        
    df.drop(columns=['_id'],inplace=True)
    
    return df.to_dict('records'), dropdown_placehoder

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('filter-type', 'value'),
        Input('reset-button', 'n_clicks')
    ]
)
def update_graphs(viewData, value, n_clicks):
    if value == 'Water Rescue':
        df = pd.DataFrame.from_records(db.water_rescue_filter())
        names_arg = 'breed'
        title_arg = 'Animal Breeds'
    if value == 'Mountain or Wilderness Rescue':
        df = pd.DataFrame.from_records(db.mountain_wilderness_rescue_filter())
        names_arg = 'breed'
        title_arg = 'Animal Breeds'
    if value == 'Disaster Rescue or Individual Tracking':
        df = pd.DataFrame.from_records(db.disaster_individual_rescue_filter())
        names_arg = 'breed'
        title_arg = 'Animal Breeds'
    if 'reset-button' == ctx.triggered_id or value is None:
        df = pd.DataFrame.from_records(db.read({}))
        names_arg = 'animal_type'
        title_arg = 'Animal Types'

    return [
       dcc.Graph(            
           figure = px.pie(df, names=names_arg, title=title_arg)
       )    
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]

    # Add no name if animal does not have name in db
    if not dff.iloc[row,9]:
        animal_name = "No name"
    else:
        animal_name = dff.iloc[row,9]

    # This is to add an image to tooltip hover depending on the animal type
    if dff.iloc[row,3] == "Dog":
        image_path = 'assets/dog.png'
    if dff.iloc[row,3] == "Cat":
        image_path = 'assets/cat.png'
    if dff.iloc[row,3] == "Other":
        image_path = 'assets/other.png'
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75,-97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   # Marker with tool tip and popup
                   # Column 13 and 14 define the grid-coordinates for 
                   # the map
                   # Column 4 defines the breed for the animal
                   # Column 9 defines the name of the animal
                   dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
                             children=[
                                 dl.Tooltip([
                                     dff.iloc[row,4],
                                     html.Br(),
                                     html.Img(src=image_path)
                                 ]),
                                 dl.Popup([
                                     html.H1("Animal Info"),
                                     html.P([
                                         html.B("Animal Name: "),
                                         animal_name,
                                         html.Br(),
                                         html.B("Animal Type: "),
                                         dff.iloc[row,3],
                                         html.Br(),
                                         html.B("Color: "),
                                         dff.iloc[row,5]
                                     ])
                                 ])
                             ])
               ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:10584/
